In [ ]:
import re

import pandas as pd

In [ ]:
with open("model.msh") as f:
    raw_dat = f.readlines()
    dat = []
    
    # 末尾の"\n"を削除
    for d in raw_dat:
        dat.append(d[:-1])

In [ ]:
nodes = []
RE_PATTERN_NODE = r"(?P<node_no>[\d]+)[\s]+(?P<node_x>[-.Ee\d]+)[\s]+(?P<node_y>[-.Ee\d]+)[\s]+(?P<node_z>[-.Ee\d]+)"
search_f = False

# Get node data
for d in dat:
    if re.match(r"\$Nodes", d):
        search_f = True
    if re.match(r"\$EndNodes", d):
        search_f = False
        break
    m = re.match(RE_PATTERN_NODE, d)
    if m and search_f:
        nodes.append([
            int(m.group("node_no")),
            float(m.group("node_x")),
            float(m.group("node_y")),
            float(m.group("node_z"))
        ])

df_nodes = pd.DataFrame(nodes, columns=("node_no", "node_x", "node_y", "node_z"))

In [ ]:
elements = []
element_no = 1
RE_COMPILE = re.compile(r"""(?P<element_no>[\d]+)[\s]+
                            (?P<class_1>[\d]+)[\s]+
                            (?P<class_2>[\d]+)[\s]+
                            (?P<class_3>[\d]+)[\s]+
                            (?P<class_4>[\d]+)[\s]+
                            (?P<node_id_1>[\d]+)[\s]+
                            (?P<node_id_2>[\d]+)[\s]+
                            (?P<node_id_3>[\d]+)
                            """, re.VERBOSE)
search_f = False

# Get node data
for d in dat:
    if re.match(r"\$Elements", d):
        search_f = True
    if re.match(r"\$EndElements", d):
        search_f = False
        break
    m = RE_COMPILE.match(d)
    if m and search_f:
        elements.append([
            element_no,
            int(m.group("node_id_1")),
            int(m.group("node_id_2")),
            int(m.group("node_id_3")),
        ])
        element_no += 1

In [ ]:
tmp = []

model_data = []

for el in elements:
    tmp.append(el[0])  # element_no
    tmp.append(el[1])  # tri_node_1
    tmp.append(el[2])  # tri_node_2
    tmp.append(el[3])  # tri_node_3
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[1]]["node_x"]))  # tri_node_1_x
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[1]]["node_y"]))  # tri_node_1_y
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[2]]["node_x"]))  # tri_node_2_x
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[2]]["node_y"]))  # tri_node_2_y
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[3]]["node_x"]))  # tri_node_3_x
    tmp.append(float(df_nodes[df_nodes["node_no"] == el[3]]["node_y"]))  # tri_node_3_y

    model_data.append(tmp)
    tmp = []

    
columns = ["element_no", "tri_node_1", "tri_node_2", "tri_node_3", "tri_node_1_x", "tri_node_1_y", "tri_node_2_x", "tri_node_2_y", "tri_node_3_x", "tri_node_3_y"]
df_model_data = pd.DataFrame(model_data, columns=columns)



In [ ]:
# 書き出し
df_model_data.to_csv("model_data.csv")
df_nodes.to_csv("node_data.csv")